In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8439.76it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4712.19it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2737950.04it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8895.89it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4797.60it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 82137.05it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 8023.85it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4869.51it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 62619.15it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 9101.69it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4820.53it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 15213.65it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
bin_size = 30
model = mz.models.DRMM()
model.params['input_shapes'] = [[10,], [10, bin_size,]]
model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 300
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DRMM.
Parameter "embedding_trainable" set to True.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 300)      5002200     text_left[0][0]                  
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10, 1)        300         embedding[0][0]                  
__________________________________________________________________________________________________
match_histogram (InputLayer)    (None, 10, 30)       0                                            
__________________________________

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
# normalize the word embedding for fast histogram generating.
l2_norm = np.sqrt((embedding_matrix*embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [9]:
model.load_embedding_matrix(embedding_matrix)

In [10]:
pred_generator = mz.HistogramDataGenerator(data_pack=predict_pack_processed,
                                           embedding_matrix=embedding_matrix,
                                           bin_size=bin_size, 
                                           hist_mode='LCH')
pred_x, pred_y = pred_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, 
                                           x=pred_x, 
                                           y=pred_y, 
                                           once_every=1, 
                                           batch_size=len(pred_y),
                                           model_save_path='./drmm_pretrained_model/'
                                          )

In [11]:
train_generator = mz.HistogramPairDataGenerator(train_pack_processed, embedding_matrix, bin_size, 'LCH',
                                                num_dup=2, num_neg=4, batch_size=20)
len(train_generator)

102

In [12]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 10s 99ms/step - loss: 1.5309
Validation: normalized_discounted_cumulative_gain@3(0): 0.4986762775699343 - normalized_discounted_cumulative_gain@5(0): 0.5504466953195465 - mean_average_precision(0): 0.5154984774159246
Epoch 2/30
102/102 [==============================] - 8s 76ms/step - loss: 1.3919
Validation: normalized_discounted_cumulative_gain@3(0): 0.5549393331491589 - normalized_discounted_cumulative_gain@5(0): 0.6141207417094355 - mean_average_precision(0): 0.5726396260841911
Epoch 3/30
102/102 [==============================] - 8s 81ms/step - loss: 1.2867
Validation: normalized_discounted_cumulative_gain@3(0): 0.573692471991038 - normalized_discounted_cumulative_gain@5(0): 0.6329777160881896 - mean_average_precision(0): 0.5860880510000691
Epoch 4/30
102/102 [==============================] - 9s 87ms/step - loss: 1.1994
Validation: normalized_discounted_cumulative_gain@3(0): 0.6033421839585815 - normalized_discounted_cumulativ

102/102 [==============================] - 7s 68ms/step - loss: 0.6053
Validation: normalized_discounted_cumulative_gain@3(0): 0.6030962757658368 - normalized_discounted_cumulative_gain@5(0): 0.6550618042545611 - mean_average_precision(0): 0.6153120002258798
Epoch 29/30
102/102 [==============================] - 7s 67ms/step - loss: 0.5999ation: normalized_discounted_cumulative_gain@3(0): 0.6030962757658368 - normalized_discounted_cumulative_gain@5(0): 0.6550618042545611 - mean_average_precision(0): 0.615312000225879
Validation: normalized_discounted_cumulative_gain@3(0): 0.6095742357064031 - normalized_discounted_cumulative_gain@5(0): 0.6609072243676714 - mean_average_precision(0): 0.6193984247841957
Epoch 30/30
102/102 [==============================] - 7s 67ms/step - loss: 0.5962
Validation: normalized_discounted_cumulative_gain@3(0): 0.6097187964978661 - normalized_discounted_cumulative_gain@5(0): 0.662263044884704 - mean_average_precision(0): 0.6186700966708153


In [13]:
drmm_model = mz.load_model('./drmm_pretrained_model/16')
test_generator = mz.HistogramDataGenerator(data_pack=valid_pack_processed[:10],
                                           embedding_matrix=embedding_matrix,
                                           bin_size=bin_size, 
                                           hist_mode='LCH')
test_x, test_y = test_generator[:]
prediction = drmm_model.predict(test_x)
prediction

array([[ 0.49939275],
       [ 2.2709463 ],
       [-3.1269906 ],
       [-0.03658384],
       [-2.9051058 ],
       [-1.0289317 ],
       [ 1.0633922 ],
       [-0.64751583],
       [ 1.0268258 ],
       [ 1.7241758 ]], dtype=float32)

In [14]:
import shutil
shutil.rmtree('./drmm_pretrained_model/')